# Tracking healthy vs. damaged mRNA transcripts

In [9]:
import tqdm
import bokeh
import numpy as np

# Column 0 is change in m, column 1 is change in p
simple_update = np.array([[1, 0, 0],    # Make Healthy mRNA transcript
                        [0, 1, 0],      # Make Damaged mRNA transcript
                        [-1, 0, 0],     # Degrade Healthy mRNA transcript
                        [0, -1, 0],     # Degrade Damaged mRNA transcript
                        [0, 0, 1],      # Accumulate damage on gene
                        [0, 0, -1],     # Repair Damage on gene
                        [0, 1, 0]       # Do not detect damage and make damaged transcript              
                        ], 
                        dtype=int)
                        
def sample_discrete(probs):
    """Randomly sample an index with probability given by probs."""
    # Generate random number
    q = np.random.rand()
    
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1 # This is returning the reaction event that will occur

def simple_propensity(propensities, population, t, beta_m, gamma_m, u, Pd):
    """Updates an array of propensities given a set of parameters
    and an array of populations.
    """
    # Unpack population
    m_healthy = population[0] # number of healthy mRNAs
    m_damaged = population[1] # number of damaged mRNAs
    d = population[2]         # number of damage sites
    
    # Update propensities
    if d == 0:
        propensities[0] = beta_m      # Make healthy mRNA transcript at rate beta_m if no damage on gene
        propensities[1] = 0           # Probability of Make damaged mRNA
        
        propensities[2] = m_healthy * gamma_m # Degrade healthy mRNA
        propensities[3] = m_damaged * gamma_m # Degrade damaged mRNA
        propensities[4] = u           # Add a damage site to the gene
        propensities[5] = 0 # Detect damage site and repair
        propensities[6] = 0 # Do not detect damage site and transcribe mRNA (with error in it)
    
    else:
        propensities[0] = 0           # if there is damage on DNA, probability of making healthy mRNA = 0
        propensities[1] = 0      # Make damaged mRNA

        propensities[2] = m_healthy * gamma_m # Degrade healthy mRNA
        propensities[3] = m_damaged * gamma_m # Degrade damaged mRNA

        propensities[4] = u           # Add a damage site to the gene
        propensities[5] = beta_m * ((1-(1-Pd)**d)) # Detect damage site and repair
        propensities[6] = beta_m * ((1-Pd)**d) # Do not detect damage site and transcribe mRNA (with error in it)
    



    
def gillespie_draw(propensity_func, propensities, population, t, args=()):
    """
    Draws a reaction and the time it took to do that reaction.
    
    Parameters
    ----------
    propensity_func : function
        Function with call signature propensity_func(population, t, *args)
        used for computing propensities. This function must return
        an array of propensities.
    population : ndarray
        Current population of particles
    t : float
        Value of the current time.
    args : tuple, default ()
        Arguments to be passed to `propensity_func`.
        
    Returns
    -------
    rxn : int
        Index of reaction that occured.
    time : float
        Time it took for the reaction to occur.
    """
    # Compute propensities
    propensity_func(propensities, population, t, *args)
    
    # Sum of propensities
    props_sum = propensities.sum()
    
    # Compute next time
    time = np.random.exponential(1.0 / props_sum)
    
    # Compute discrete probabilities of each reaction
    rxn_probs = propensities / props_sum
    
    # Draw reaction from this distribution
    rxn = sample_discrete(rxn_probs)
    
    return rxn, time



def gillespie_ssa(propensity_func, update, population_0, time_points, args=()):
    """
    Uses the Gillespie stochastic simulation algorithm to sample
    from probability distribution of particle counts over time.
    
    Parameters
    ----------
    propensity_func : function
        Function of the form f(params, t, population) that takes the current
        population of particle counts and return an array of propensities
        for each reaction.
    update : ndarray, shape (num_reactions, num_chemical_species)
        Entry i, j gives the change in particle counts of species j
        for chemical reaction i.
    population_0 : array_like, shape (num_chemical_species)
        Array of initial populations of all chemical species.
    time_points : array_like, shape (num_time_points,)
        Array of points in time for which to sample the probability
        distribution.
    args : tuple, default ()
        The set of parameters to be passed to propensity_func.        

    Returns
    -------
    sample : ndarray, shape (num_time_points, num_chemical_species)
        Entry i, j is the count of chemical species j at time
        time_points[i].
    """

    # Initialize output
    pop_out = np.empty((len(time_points), 3), dtype=int) # the length always stays the same as the time points, but the second parameterr
                                                         # must match the amount of variables we are keeping track of

    # Initialize and perform simulation
    i_time = 1
    i = 0
    t = time_points[0]
    population = population_0.copy()
    pop_out[0,:] = population
    propensities = np.zeros(update.shape[0])
    while i < len(time_points):
        while t < time_points[i_time]:
            # draw the event and time step
            event, dt = gillespie_draw(propensity_func, propensities, population, t, args)
                
            # Update the population
            population_previous = population.copy()
            population += update[event]

            # can't have negative damage on the gene
            if population[2] < 0: # this needs to match whatever index I am keeping track of the damage at in population
                population[2] = 0
            
            # Increment time
            t += dt

        # Update the index
        i = np.searchsorted(time_points > t, True)
        
        # Update the population
        pop_out[i_time:min(i,len(time_points))] = population_previous
        
        # Increment index
        i_time = i
                        
    return pop_out 

This served as a good base, model but some things I still don't fully understand, such as the dimensionless time aspect. If this is dimensionless, I am unsure how to get proper beta and gamma coefficients considering theres no time.

The sedimentation profiles of mRNA from HeLa and CHO cells indicate that the lengths of mammalian mRNA are lognormally distributed with a median value of 1.4 kb and a deviation of 2.0. This implies that, on the average, a mRNA species is only about 25% larger than the mature polypeptide it codes for. [https://link.springer.com/article/10.1007/BF01732582]

Could use this result to but in a good value for $\beta_m$
Could tailor this rate to deal with certain specific proteins we know the mRNA length of?
Maybe deal with the degradation rate in a similar way? [https://pmc.ncbi.nlm.nih.gov/articles/PMC403777/]

Unsure if this is the right direction to be going in for the model.


In [10]:
# Specify parameters for calculation
beta_mrna = 10.0 # transcription rate
gamma_mrna = 0.1 # degradation rate
u = 0.0 # damage accumulation rate
Pd = 0.0 # rate of damage detection

args = (beta_mrna, gamma_mrna, u, Pd)
time_points = np.linspace(0, 500, 1001)
population_0 = np.array([0, 0, 0], dtype=int)
size = 10

# Seed random number generator for reproducibility
np.random.seed(42)

# Initialize output array
samples = np.empty((size, len(time_points), 3), dtype=int) # again, change the number here to the amount of things we are tracking

# shape of samples is (size, time_points, amount of species)
# size = amount of times simulation is run
# time_points = amount of time points in each simulation
# amount of species = how many things we are tracking, mRNA, amount of damage on gene, etc.

# Run the calculations
for i in tqdm.tqdm_notebook(range(size)):
    # this is storing the population at each time point for each simulation being ran (i)
    samples[i,:,:] = gillespie_ssa(simple_propensity, simple_update,
                                population_0, time_points, args=args)

C:\Users\joeot\AppData\Local\Temp\ipykernel_19304\715350819.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(size)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
import bokeh.plotting as bp
from bokeh.io import show, output_notebook
from bokeh.models import Span

output_notebook()
# Set up plot for mRNA only
plot = bp.figure(width=600,
                 height=400,
                 title=f"mRNA counts with Beta {beta_mrna}, Gamma {gamma_mrna}",
                 x_axis_label='dimensionless time',
                 y_axis_label='number of mRNAs')

# Plot multiple trajectories
for x in samples[:, :, 0]:
    plot.line(time_points, x, line_width=0.3, alpha=0.2, line_join='bevel')

# Plot the mean trajectory
plot.line(time_points, samples[:, :, 0].mean(axis=0),
          line_width=4, color='orange', line_join='bevel', legend_label="mRNA Counts")

steady_state = beta_mrna / gamma_mrna

# Horizontal line
hline = Span(location=steady_state, dimension='width', line_color='red', line_width=2)

plot.renderers.extend([hline])

show(plot)

Loading BokehJS ...

In [12]:
plots = [bp.figure(width=600,
                 height=400,
                 title=f"mRNA counts with Beta {beta_mrna}, Gamma {gamma_mrna}",
                 x_axis_label='dimensionless time',
                 y_axis_label='number of Healthy mRNAs'),

         bp.figure(width=600,
                               height=400,
                               x_axis_label='dimensionless time',
                               y_axis_label='number of Damaged mRNAs'),
                                
        bp.figure(width=600,
                               height=400,
                               x_axis_label='dimensionless time',
                               y_axis_label='Damage sites on Gene'),
        
        bp.figure(width=600,
                               height=400,
                               x_axis_label='dimensionless time',
                               y_axis_label='Total mRNAs')
                               ]

# Plot trajectories and mean
for i in range(0, 3): # this should loop over all the plots I want to make
    for x in samples[:,:,i]: # when i = 0, we are plotting population[0]
                             # when i = 1, we are plotting population[1] etc.
        
        # plot the individual simulation trajectories
        plots[i].line(time_points, x, line_width=0.3, 
                      alpha=0.2, line_join='bevel')
        
    
    # this plots the mean path 
    plots[i].line(time_points, samples[:,:,i].mean(axis=0),
                  line_width=1, color='orange', line_join='bevel')

# Making my total plot, I could probably do something with indices here and avoid
# three separate loops, but this seems to work and speeds not an issue right now
for x in samples[:,:,0]:
    plots[3].line(time_points, x, line_width=0.3, alpha=0.2, line_join='bevel', color='blue') # Healthy mRNAs

for y in samples[:,:,1]:
    plots[3].line(time_points, y, line_width=0.3, alpha=0.2, line_join='bevel', color='red') # Damaged mRNAs

for z in (samples[:,:,0] + samples[:,:,1]):
    plots[3].line(time_points, z, line_width=0.3, alpha=0.2, line_join='bevel', color='purple') # total mRNAs

plots[3].line(time_points, samples[:,:,0].mean(axis=0), line_width=1, color='blue', line_join='bevel') # Average healthy mRNAs
plots[3].line(time_points, samples[:,:,1].mean(axis=0), line_width=1, color='red', line_join='bevel') # Average damaged mRNAs
plots[3].line(time_points, (samples[:,:,0] + samples[:, :, 1]).mean(axis=0), line_width=1, color='purple', line_join='bevel') # Average total mRNAs
    


# Link axes
#plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=4))